In [69]:
import pandas as pd
import numpy as np

In [70]:
# Path
metadata_path = 'metadata.csv'
adj_matrix_path = 'Adjacency Matrix.csv'

# Ouput Path
adj_matrix_output_path = 'Adjacency_Matrix_Preprocess.csv'

In [71]:
meta_df = pd.read_csv(metadata_path)
adj_matrix_df = pd.read_csv(adj_matrix_path)

In [72]:
meta_df.head()

,Node,Pos_X,Pos_Y,Destination
0,1,879,4924,Parabol Gate
1,2,1790,4924,Material Base
2,3,2106,4924,Material Base
3,4,2918,4924,Viet-Duc Building
4,5,3944,4924,TQB-Library


In [73]:
adj_matrix_df.head()

,Unnamed: 0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,1.0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2.0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,3.0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4.0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,5.0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [74]:
adj_matrix_df.tail()

,Unnamed: 0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
31,32.0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
32,33.0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33,34.0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
34,35.0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
35,NaN,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35


In [75]:
adj_matrix_df = adj_matrix_df.iloc[:adj_matrix_df.shape[0]-1, :]
adj_matrix_df.tail()

,Unnamed: 0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
30,31.0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
31,32.0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
32,33.0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33,34.0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
34,35.0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [76]:
print(adj_matrix_df.shape)

(35, 36)


In [77]:
row_count = adj_matrix_df.shape[0]
column_count = adj_matrix_df.shape[1]
print(row_count)
print(column_count)

35
36


In [78]:
# Test extract data from meta_df
node_test = meta_df[meta_df['Node']==1]
print(node_test)
print(type(node_test['Pos_X']))
print(node_test['Pos_X'].values[0])
print(node_test['Pos_Y'].values[0])

   Node  Pos_X  Pos_Y   Destination
0     1    879   4924  Parabol Gate
<class 'pandas.core.series.Series'>
879
4924


In [79]:
#Utils

# Compute Weight - Euclidean Distance
def compute_norm_2(point1 : tuple, point2 : tuple):
    dx = (point2[0] - point1[0])**2
    dy = (point2[1] - point1[1])**2
    return (dx + dy)**(1/2)

def compute_weight(node_num_1, node_num_2, _meta_data):
    node_1 = _meta_data[_meta_data['Node']==node_num_1]
    node_2 = _meta_data[_meta_data['Node']==node_num_2]
    
    node_1_x = node_1['Pos_X'].values[0]
    node_1_y = node_1['Pos_Y'].values[0]
    node_2_x = node_2['Pos_X'].values[0]
    node_2_y = node_2['Pos_Y'].values[0]
    
    return compute_norm_2((node_1_x, node_1_y), (node_2_x, node_2_y))

# test
print(compute_weight(1, 2, meta_df))

911.0


In [80]:
for i in range(row_count):
    for j in range(1, column_count):
        if adj_matrix_df.iloc[i, j] == 1:
            adj_matrix_df.iloc[i, j] = compute_weight(i+1, j, meta_df)

adj_matrix_df.head()

,Unnamed: 0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,1.0,0.0,911.0,0.0,0.0,0,0.0,0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,2.0,911.0,0.0,316.0,0.0,0,0.0,0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1076.078529
2,3.0,0.0,316.0,0.0,812.0,0,0.0,0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
3,4.0,0.0,0.0,812.0,0.0,1026,0.0,0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1069.105233,0.000000
4,5.0,0.0,0.0,0.0,1026.0,0,1320.0,0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000


In [81]:
adj_matrix_df_sum = np.sum(adj_matrix_df.iloc[:, 1:].to_numpy())
print(adj_matrix_df_sum)

82336.02988969714


In [82]:
multi_ceof
for i in range(row_count):
    for j in range(1, column_count):
        if adj_matrix_df.iloc[i, j] != 0:
            adj_matrix_df.iloc[i, j] = round(adj_matrix_df.iloc[i, j]/adj_matrix_df_sum, 5)
        else:
            adj_matrix_df.iloc[i, j] = round(adj_matrix_df.iloc[i, j])

adj_matrix_df.head()

,Unnamed: 0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,1.0,0.00000,0.01106,0.00000,0.00000,0.00000,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000
1,2.0,0.01106,0.00000,0.00384,0.00000,0.00000,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.01307
2,3.0,0.00000,0.00384,0.00000,0.00986,0.00000,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000
3,4.0,0.00000,0.00000,0.00986,0.00000,0.01246,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01298,0.00000
4,5.0,0.00000,0.00000,0.00000,0.01246,0.00000,0.01603,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000


In [83]:
adj_matrix_df.head()

,Unnamed: 0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,1.0,0.00000,0.01106,0.00000,0.00000,0.00000,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000
1,2.0,0.01106,0.00000,0.00384,0.00000,0.00000,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.01307
2,3.0,0.00000,0.00384,0.00000,0.00986,0.00000,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000
3,4.0,0.00000,0.00000,0.00986,0.00000,0.01246,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01298,0.00000
4,5.0,0.00000,0.00000,0.00000,0.01246,0.00000,0.01603,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000


In [87]:
adj_matrix_df.tail()

,Unnamed: 0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
30,31.0,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0000,0.0264,0.0,0.0,0.0,0.0,0.00000,0.00000
31,32.0,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0253,0.0000,0.0,0.0,0.0,0.0,0.00000,0.00000
32,33.0,0.0,0.00000,0.0,0.00000,0.0,0.01845,0.0,0.0,0.0,...,0.0,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.00000,0.00000
33,34.0,0.0,0.00000,0.0,0.01298,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.00000,0.01372
34,35.0,0.0,0.01307,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.01372,0.00000


In [86]:
adj_matrix_df.iloc[:, 1:].describe()

,1,2,3,4,5,6,7,8,9,10,...,26,27,28,29,30,31,32,33,34,35
count,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,...,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000
mean,0.001307,0.000799,0.000820,0.001009,0.000814,0.001743,0.000983,0.001179,0.001063,0.000576,...,0.000219,0.000471,0.001544,0.001737,0.000365,0.000754,0.000723,0.000527,0.000763,0.000765
std,0.006100,0.002898,0.003038,0.003367,0.003383,0.005249,0.004114,0.004078,0.003733,0.002658,...,0.001298,0.001961,0.005621,0.005994,0.002160,0.004462,0.004276,0.003119,0.003145,0.003156
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.034680,0.013070,0.015010,0.012980,0.016030,0.020120,0.020120,0.018290,0.018290,0.015010,...,0.007680,0.009370,0.025300,0.026400,0.012780,0.026400,0.025300,0.018450,0.013720,0.013720


In [90]:
adj_matrix_df.to_csv(adj_matrix_output_path, index = False)